<a href="https://colab.research.google.com/github/catmisty/aarthy_litterdetection/blob/main/Aarthy_litterdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# collecting image

mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


TACO

In [ ]:
!git clone https://github.com/pedropro/TACO.git
%cd TACO


Cloning into 'TACO'...
remote: Enumerating objects: 740, done.
remote: Counting objects: 100% (435/435), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 740 (delta 416), reused 380 (delta 380), pack-reused 305 (from 1)
Receiving objects: 100% (740/740), 97.48 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (499/499), done.
/content/yolov5/TACO


Download the TACO dataset

In [ ]:
!python download.py

Note. If for any reason the connection is broken. Just call me again and I will start where I left.
Finished


In [ ]:
!pip install -r requirements.txt
!pip install torch torchvision torchaudio
!pip install tensorflow

Set Up YOLOv5 for Training

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16954, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 16954 (delta 76), reused 96 (delta 46), pack-reused 16805 (from 1)
Receiving objects: 100% (16954/16954), 15.70 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (11605/11605), done.
/content/yolov5


In [ ]:
!pip install -r requirements.txt

Convert TACO Annotations to YOLO Format

In [ ]:
import json
import os

# Load the TACO annotations
with open('/content/TACO/data/annotations.json') as f:
    taco_data = json.load(f)

output_dir = '/content/TACO/labels/'
os.makedirs(output_dir, exist_ok=True)

# Function to convert TACO bbox format to YOLO format
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_center = box[0] + box[2] / 2.0
    y_center = box[1] + box[3] / 2.0
    width = box[2]
    height = box[3]
    return x_center * dw, y_center * dh, width * dw, height * dh

# Process each image annotation
for ann in taco_data['annotations']:
    image_id = ann['image_id']
    image = next(img for img in taco_data['images'] if img['id'] == image_id)

    # Open the corresponding label file
    label_file = os.path.join(output_dir, f"{image_id}.txt")
    with open(label_file, 'a') as lf:
        # Convert the bbox to YOLO format
        bbox = convert_bbox((image['width'], image['height']), ann['bbox'])
        category_id = ann['category_id'] - 1  # YOLO classes start from 0
        # Write to the label file: class_id x_center y_center width height
        lf.write(f"{category_id} {' '.join(map(str, bbox))}\n")


In [ ]:
!ls /content/TACO/

data  demo.ipynb  detector  download.py  labels  LICENSE  README.md  requirements.txt  yolov5


In [ ]:
!ls /content/TACO/data/

all_image_urls.csv  annotations_unofficial.json  batch_10  batch_12
annotations.json    batch_1			 batch_11  batch_13


Organize the Dataset

In [ ]:
!mkdir -p /content/TACO/images/train /content/TACO/images/val /content/TACO/labels/train /content/TACO/labels/val


In [ ]:
import os

os.makedirs('/content/TACO/images/train', exist_ok=True)
os.makedirs('/content/TACO/images/val', exist_ok=True)


In [ ]:
import shutil
import random
from glob import glob
import os

# Create directories for train and val sets
os.makedirs('/content/TACO/images/train', exist_ok=True)
os.makedirs('/content/TACO/images/val', exist_ok=True)
os.makedirs('/content/TACO/labels/train', exist_ok=True)
os.makedirs('/content/TACO/labels/val', exist_ok=True)

# Get a list of all batch directories
batch_dirs = glob('/content/TACO/data/batch_*')

image_files = []
for batch_dir in batch_dirs:
    # Get a list of all images in each batch directory
    image_files.extend(glob(os.path.join(batch_dir, '*.jpg')))

# Shuffle and split images into train/val
random.shuffle(image_files)
train_size = int(0.8 * len(image_files))
train_files = image_files[:train_size]
val_files = image_files[train_size:]

# Function to rename and move files to avoid overwriting
def move_files(file_list, dest_dir):
    for file in file_list:
        base_name = os.path.basename(file)
        dest_file = os.path.join(dest_dir, base_name)

        # Rename file if a file with the same name already exists
        counter = 1
        while os.path.exists(dest_file):
            name, ext = os.path.splitext(base_name)
            dest_file = os.path.join(dest_dir, f"{name}_{counter}{ext}")
            counter += 1

        print(f"Moving {file} to {dest_file}")  # Debugging statement
        shutil.move(file, dest_file)

# Move the files to train/val directories
move_files(train_files, '/content/TACO/images/train/')
move_files(val_files, '/content/TACO/images/val/')

# Function to rename and move labels to avoid overwriting
def move_labels(file_list, dest_dir):
    for file in file_list:
        label_file = file.replace('.jpg', '.txt')
        if os.path.exists(label_file):
            base_name = os.path.basename(label_file)
            dest_file = os.path.join(dest_dir, base_name)

            # Rename file if a file with the same name already exists
            counter = 1
            while os.path.exists(dest_file):
                name, ext = os.path.splitext(base_name)
                dest_file = os.path.join(dest_dir, f"{name}_{counter}{ext}")
                counter += 1

            print(f"Moving {label_file} to {dest_file}")  # Debugging statement
            shutil.move(label_file, dest_file)

# Move the labels to corresponding train/val directories
move_labels(train_files, '/content/TACO/labels/train/')
move_labels(val_files, '/content/TACO/labels/val/')


In [ ]:
# Function to rename and move labels to avoid overwriting
def move_labels(file_list, dest_dir):
    for file in file_list:
        label_file = file.replace('.jpg', '.txt')
        if os.path.exists(label_file):
            base_name = os.path.basename(label_file)
            dest_file = os.path.join(dest_dir, base_name)

            # Rename file if a file with the same name already exists
            counter = 1
            while os.path.exists(dest_file):
                name, ext = os.path.splitext(base_name)
                dest_file = os.path.join(dest_dir, f"{name}_{counter}{ext}")
                counter += 1

            shutil.move(label_file, dest_file)

# Move the labels to corresponding train/val directories
move_labels(train_files, '/content/TACO/labels/train/')
move_labels(val_files, '/content/TACO/labels/val/')


In [ ]:
data_yaml = """
train: /content/TACO/images/train/
val: /content/TACO/images/val/
test: /content/TACO/images/test/

nc: 10  # Number of classes in TACO
names: ['Plastic', 'Paper', 'Metal', 'Glass', 'Trash', 'Organic Waste', 'Other', 'Rubber', 'Textile', 'Wood']
"""

with open('/content/yolov5/data/taco.yaml', 'w') as f:
    f.write(data_yaml)


In [ ]:
!ls /content/TACO/images/train

In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
!ls /content/TACO/images/train

In [ ]:
!ls /content/yolov5/data/taco.yaml

/content/yolov5/data/taco.yaml


Train the YOLOv5 Model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/yolov5/data/taco.yaml --weights yolov5s.pt

2024-09-15 09:16:28.847217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 09:16:28.882298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 09:16:28.892809: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/taco.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, si